In [1]:
import os
import time
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [10]:
# --------------- CONFIGURAÇÃO DE DOWNLOAD ---------------

def criar_pasta_download():
    base_dir = os.getcwd()  # pasta onde o script está sendo executado
    download_dir = os.path.join(base_dir, "csvs_prefeitos")

    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    return download_dir


def criar_driver(download_dir: str):
    """Cria o ChromeDriver já configurado pra baixar na pasta download_dir."""
    options = webdriver.ChromeOptions()

    prefs = {
        "download.default_directory": download_dir,      # pasta de download
        "download.prompt_for_download": False,          # sem pop-up de download
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)

    # Opcional: rodar sem abrir janela
    # options.add_argument("--headless=new")

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

# --------------- KEYWORDS ---------------
def wait_until_visible(driver, locator, timeout=20):
    """
    locator = (By.CSS_SELECTOR, ".classe")
    """
    try:
        WebDriverWait(driver, timeout).until(
            EC.visibility_of_element_located(locator)
        )
        return True
    except:
        return False

# --------------- LÓGICA DE CLIQUE (VOCÊ COMPLETA) ---------------

def clicar_botoes_para_baixar(driver):
    """
    Aqui você coloca os cliques nos botões que disparam os downloads dos CSVs.
    Ex: encontrar botões, percorrer lista, clicar um por um etc.   //*[@id="candidato-data"]/div[2]/div[3]/div[2]
    """

    cards = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[2]/div/div[3]')
    total = len(cards)
    
    turno_1 = '//*[@id="candidato-data"]/div[2]/div[3]/div[1]/input'
    turno_2 = '//*[@id="candidato-data"]/div[2]/div[3]/div[2]/input'

    for i in range(total):
        cards[i].click()
        turno_visivel = wait_until_visible(driver, (By.XPATH, turno_1))
        if turno_visivel:
            elemento_turno = driver.find_element(By.XPATH, turno_1)
            elemento_turno.click()
            
    time.sleep(10)

# --------------- SCRIPT PRINCIPAL ---------------

def main():
    download_dir = criar_pasta_download()
    driver = criar_driver(download_dir)

    try:
        # Coloque aqui a URL da página de onde você baixa os CSVs
        url = "https://apps.tre-ce.jus.br/tre/consulta-votacao/2045202024/13897/11"
        driver.get(url)

        clicar_botoes_para_baixar(driver)


    finally:
        # Se quiser manter o navegador aberto, comente esta linha
        driver.quit()


main()


In [ ]:
cards = driver.find_elements(By.CSS_SELECTOR, "a.ui.card")
turno_1 = '//*[@id="candidato-data"]/div[2]/div[3]/div[1]/input'
turno_2 = '//*[@id="candidato-data"]/div[2]/div[3]/div[2]/input'

for card in cards:
    card.click()
    turno_visivel = wait_until_visible(driver, (By.XPATH, turno_1))
    if turno_visivel:
        elemento_turno = driver.find_element(By.XPATH, turno_1)
        elemento_turno.clear()
        elemento_turno.send_keys("1")

In [22]:
import os
import time
import traceback
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ==== CONFIGURAÇÃO DE PASTA DE DOWNLOAD ====
download_dir = os.path.abspath("csvs_prefeitos")
os.makedirs(download_dir, exist_ok=True)

# ==== CONFIGURAÇÃO DO CHROME ====
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
#options.add_argument("--headless")  # Rode sem abrir o navegador (opcional)

# ==== ABRE NAVEGADOR ====
driver = webdriver.Chrome(options=options)

# ==== URL base ====
URL = "https://apps.tre-ce.jus.br/tre/consulta-votacao/2045202024/13897/11"
driver.get(URL)

erros= []

try:
    # Espera os cards carregarem
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.ui.card"))
    )

    # Pega todos os links de cards
    cards = driver.find_elements(By.CSS_SELECTOR, "a.ui.card")
    total = len(cards)

    print(f"🔍 Encontrados {total} candidatos.")
    
    for i in range(total):
        try:
            blocos = f'//*[@id="root"]/div/div[2]/div/div[3]/a[{i}]'
            print(f"\n➡️ Processando candidato {i+1}/{total}...")

            # Recarrega a página toda vez para evitar stale elements
            driver.get(URL)
            bloco = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, blocos))
                    )
            bloco.click()

            # Espera botão e clica em "Votação por seção"
            botao_secao = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Votação por Seção')]"))
            )
            time.sleep(1)
            botao_secao.click()

            # Espera botão de download e clica
            botao_csv = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ui.green.tiny.left.attached.button.ml-2"))
            )
            botao_csv.click()

            # Aguarda o download terminar
            time.sleep(5)

            # Nome do candidato
            nome_completo = driver.find_element(By.CLASS_NAME, "custom-header-card").text
            nome_candidato = nome_completo.split(" - ")[1].strip().replace("\n", " ")
            nome_candidato = re.sub(r'[\\/*?:"<>|\n]', '', nome_candidato)

            # Renomear último arquivo baixado
            arquivos = sorted(
                os.listdir(download_dir),
                key=lambda x: os.path.getmtime(os.path.join(download_dir, x)),
                reverse=True
            )
            arquivo_baixado = arquivos[0]
            novo_nome = f"{nome_candidato}.csv"
            os.rename(os.path.join(download_dir, arquivo_baixado), os.path.join(download_dir, novo_nome))

            print(f"✅ CSV salvo como: {novo_nome}")
            
            

        except Exception as e:
            print(f"⚠️ Erro ao processar candidato {i+1}: {e}")
            erros.append(i+1)
            traceback.print_exc()
            continue

except Exception as e:
    print(f"❌ Erro geral: {e}")
    traceback.print_exc()

finally:
    driver.quit()
    print("\n🏁 Processo finalizado.")
    print(list(erros))


🔍 Encontrados 9 candidatos.

➡️ Processando candidato 1/9...
⚠️ Erro ao processar candidato 1: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff65e41a235
	0x7ff65e172630
	0x7ff65df016dd
	0x7ff65df5a27e
	0x7ff65df5a58c
	0x7ff65dfaed77
	0x7ff65dfababa
	0x7ff65df4b0ed
	0x7ff65df4bf63
	0x7ff65e445d60
	0x7ff65e43fe8a
	0x7ff65e461005
	0x7ff65e18d71e
	0x7ff65e194e1f
	0x7ff65e17b7c4
	0x7ff65e17b97f
	0x7ff65e1618e8
	0x7ffd9c3fe8d7
	0x7ffd9cf4c53c


➡️ Processando candidato 2/9...


Traceback (most recent call last):
  File "C:\Users\jpkab\AppData\Local\Temp\ipykernel_8364\3739846592.py", line 53, in <module>
    bloco = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, blocos))
            )
  File "c:\Users\jpkab\OneDrive\Desktop\Proj_vereadores\.venv\Lib\site-packages\selenium\webdriver\support\wait.py", line 138, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff65e41a235
	0x7ff65e172630
	0x7ff65df016dd
	0x7ff65df5a27e
	0x7ff65df5a58c
	0x7ff65dfaed77
	0x7ff65dfababa
	0x7ff65df4b0ed
	0x7ff65df4bf63
	0x7ff65e445d60
	0x7ff65e43fe8a
	0x7ff65e461005
	0x7ff65e18d71e
	0x7ff65e194e1f
	0x7ff65e17b7c4
	0x7ff65e17b97f
	0x7ff65e1618e8
	0x7ffd9c3fe8d7
	0x7ffd9cf4c53c



⚠️ Erro ao processar candidato 2: [WinError 183] Não é possível criar um arquivo já existente: 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\trece-ele2024-votacao-FORTALEZA-2T-SECAO-22-60002029263.csv' -> 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\ANDRÉ FERNANDES 2º turno ANDRE FERNANDES DE MOURA.csv'

➡️ Processando candidato 3/9...


Traceback (most recent call last):
  File "C:\Users\jpkab\AppData\Local\Temp\ipykernel_8364\3739846592.py", line 87, in <module>
    os.rename(os.path.join(download_dir, arquivo_baixado), os.path.join(download_dir, novo_nome))
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileExistsError: [WinError 183] Não é possível criar um arquivo já existente: 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\trece-ele2024-votacao-FORTALEZA-2T-SECAO-22-60002029263.csv' -> 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\ANDRÉ FERNANDES 2º turno ANDRE FERNANDES DE MOURA.csv'


⚠️ Erro ao processar candidato 3: [WinError 183] Não é possível criar um arquivo já existente: 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\trece-ele2024-votacao-FORTALEZA-1T-SECAO-44-60001939602.csv' -> 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\CAPITÃO WAGNER WAGNER SOUSA GOMES.csv'

➡️ Processando candidato 4/9...


Traceback (most recent call last):
  File "C:\Users\jpkab\AppData\Local\Temp\ipykernel_8364\3739846592.py", line 87, in <module>
    os.rename(os.path.join(download_dir, arquivo_baixado), os.path.join(download_dir, novo_nome))
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileExistsError: [WinError 183] Não é possível criar um arquivo já existente: 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\trece-ele2024-votacao-FORTALEZA-1T-SECAO-44-60001939602.csv' -> 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\CAPITÃO WAGNER WAGNER SOUSA GOMES.csv'


⚠️ Erro ao processar candidato 4: [WinError 183] Não é possível criar um arquivo já existente: 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\trece-ele2024-votacao-FORTALEZA-1T-SECAO-21-60002129533.csv' -> 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\CHICO MALTA FRANCISCO RAIMUNDO MALTA DE ARAUJO.csv'

➡️ Processando candidato 5/9...


Traceback (most recent call last):
  File "C:\Users\jpkab\AppData\Local\Temp\ipykernel_8364\3739846592.py", line 87, in <module>
    os.rename(os.path.join(download_dir, arquivo_baixado), os.path.join(download_dir, novo_nome))
    ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileExistsError: [WinError 183] Não é possível criar um arquivo já existente: 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\trece-ele2024-votacao-FORTALEZA-1T-SECAO-21-60002129533.csv' -> 'c:\\Users\\jpkab\\OneDrive\\Desktop\\Proj_vereadores\\Fortaleza\\csvs_prefeitos\\CHICO MALTA FRANCISCO RAIMUNDO MALTA DE ARAUJO.csv'


✅ CSV salvo como: EDUARDO GIRÃO LUIS EDUARDO GRANGEIRO GIRÃO.csv

➡️ Processando candidato 6/9...
✅ CSV salvo como: EVANDRO LEITÃO 2º turno EVANDRO SA BARRETO LEITAO.csv

➡️ Processando candidato 7/9...
✅ CSV salvo como: GEORGE LIMA GEORGE LIMA DE ARAÚJO.csv

➡️ Processando candidato 8/9...
✅ CSV salvo como: JOSÉ SARTO JOSE SARTO NOGUEIRA MOREIRA.csv

➡️ Processando candidato 9/9...
✅ CSV salvo como: TECIO NUNES TECIO NUNES SALGADO.csv

🏁 Processo finalizado.
[1, 2, 3, 4]
